### **Planejamento de Produção com Aprendizado por Reforço**

#### Introdução
Neste notebook, usamos o aprendizado por reforço para resolver um problema de sequenciamento de produção em uma fábrica.

Temos quatro linhas de produção e 20 produtos, cada um com demanda mensal específica. O objetivo é otimizar a produção para
atender às demandas minimizando custos relacionados ao estoque e ao não atendimento.

#### Como Funciona o Algoritmo de Aprendizado por Reforço
O **aprendizado por reforço (RL)** é uma abordagem onde um agente aprende a tomar decisões em um ambiente para maximizar uma recompensa cumulativa. No contexto deste problema:

- **Estado:** Representa a situação atual do estoque e produção de cada produto.
- **Ação:** Decisão sobre qual linha de produção utilizar para cada produto.
- **Recompensa:** Calculada com base em como a ação atende à demanda e minimiza o excesso de estoque.

O agente aprende uma política (estratégia de decisão) ótima através da interação repetida com o ambiente, ajustando suas ações para maximizar a recompensa acumulada ao longo do tempo. No caso deste problema, a recompensa é maximizada ao atender à demanda e minimizar custos de estoque.

#### Carregar dados de entrada
Dados de planilhas Excel para demandas, estoque inicial, produtividade e capacidades.

In [1]:
import pandas as pd
import numpy as np

# Caminho para o arquivo Excel
excel_file_path = 'input/production_data.xlsx'

# Carregar os dados de diferentes planilhas
demands_data = pd.read_excel(excel_file_path, sheet_name='Demandas')
initial_inventory_data = pd.read_excel(excel_file_path, sheet_name='Inventário Inicial')
productivity_data = pd.read_excel(excel_file_path, sheet_name='Produtividade')
line_capacities_data = pd.read_excel(excel_file_path, sheet_name='Capacidades')

# Converter os dados para numpy arrays
demands_array = demands_data.drop(columns='Product').to_numpy()
initial_inventory_array = initial_inventory_data['Estoque Inicial'].to_numpy()
productivity_array = productivity_data.drop(columns='Product').to_numpy()
line_capacities_array = line_capacities_data['Capacidade'].to_numpy()

#### Definir parâmetros e funções de suporte
Parâmetros do problema e definimos funções como a de recompensa.

In [5]:
# Parâmetros do problema
num_lines = productivity_array.shape[1]
num_products = demands_array.shape[0]
planning_horizon = demands_array.shape[1]
learning_rate = 0.1
discount_factor = 0.9
exploration_rate = 0.1
episode_qty = 10000

# Inicialização do Q-Table
q_table = np.zeros((num_products, num_lines))

# Função de recompensa
def calculate_reward(product_index, line_index, inventory, month_index, line_production_totals):
    product, line = product_index, line_index
    demand = demands_array[product, month_index]
    max_production = productivity_array[product, line] * line_capacities_array[line]
    available_capacity = line_capacities_array[line] - line_production_totals[line]
    actual_production = min(max_production, demand + inventory[product], available_capacity)
    
    total_supply = inventory[product] + actual_production
    unmet_demand = max(0, demand - total_supply)
    excess_inventory = max(0, total_supply - demand)
    
    # Penalidades e recompensas
    penalty_unmet_demand = unmet_demand * 10  # Penalidade por demanda não atendida
    penalty_excess_inventory = excess_inventory * 2  # Penalidade por excesso de estoque
    reward_fulfilled_demand = min(demand, total_supply) * 5  # Recompensa por atender à demanda
    
    return reward_fulfilled_demand - penalty_unmet_demand - penalty_excess_inventory, actual_production

# Política epsilon-greedy
def select_action_epsilon_greedy(state_index, epsilon=exploration_rate):
    if np.random.rand() < epsilon:
        return np.random.randint(0, num_lines)
    else:
        return np.argmax(q_table[state_index])

#### Implementar o algoritmo de Q-Learning
Treinamento e armazenamento dos resultados.

In [6]:
# Treinamento Q-Learning
for episode in range(episode_qty):
    inventory = initial_inventory_array.copy()
    for month in range(planning_horizon):
        line_production_totals = np.zeros(num_lines)
        for product_index in range(num_products):
            state = product_index
            action = select_action_epsilon_greedy(state)
            
            # Calcular a produção respeitando a capacidade disponível
            reward, production = calculate_reward(state, action, inventory, month, line_production_totals)
            
            # Ajustar produção se exceder a capacidade disponível na linha
            available_capacity = line_capacities_array[action] - line_production_totals[action]
            production = min(production, available_capacity)  # Ajuste final para respeitar a capacidade
            
            # Atualizar produção total por linha e estoque
            line_production_totals[action] += production
            inventory[product_index] = max(0, inventory[product_index] + production - demands_array[product_index, month])
            
            next_state = (state + 1) % num_products
            next_action = np.argmax(q_table[next_state])
            
            # Atualização do Q-Table
            q_table[state, action] += learning_rate * (reward + discount_factor * q_table[next_state, next_action] - q_table[state, action])

#### Gerar e exibir o resultado final
Resultados em um DataFrame para visualização.

In [7]:
# Determinar a melhor política
optimal_policy = np.argmax(q_table, axis=1)
production_plan = []

# Verificar total de produção por linha por mês
corrected_line_production_totals = np.zeros((num_lines, planning_horizon))

# Populando o total de produção por linha por mês corrigido
corrected_production_schedule = []

for product_index in range(num_products):
    assigned_line = optimal_policy[product_index]
    inventory = initial_inventory_array[product_index]
    monthly_data = {
        'Product': product_index,
        'Assigned Line': assigned_line,
        'Initial Inventory': [],
        'Production': [],
        'Demand': [],
        'Final Inventory': []
    }
    for month_index in range(planning_horizon):
        monthly_data['Initial Inventory'].append(inventory)
        demand = demands_array[product_index, month_index]
        max_production = productivity_array[product_index][assigned_line] * line_capacities_array[assigned_line]
        available_capacity = line_capacities_array[assigned_line] - corrected_line_production_totals[assigned_line, month_index]
        production = min(max_production, demand + inventory, available_capacity)
        
        # Ajustar a produção para garantir que não exceda a capacidade disponível
        production = min(production, line_capacities_array[assigned_line] - corrected_line_production_totals[assigned_line, month_index])
        
        corrected_line_production_totals[assigned_line, month_index] += production
        final_inventory = max(0, inventory + production - demand)
        
        monthly_data['Production'].append(production)
        monthly_data['Demand'].append(demand)
        monthly_data['Final Inventory'].append(final_inventory)
        
        inventory = final_inventory
    
    corrected_production_schedule.append(monthly_data)

# Criando o DataFrame com os resultados
output_records = []
for schedule_item in corrected_production_schedule:
    for var in ['Initial Inventory', 'Production', 'Demand', 'Final Inventory']:
        record = [schedule_item['Product'], schedule_item['Assigned Line'], var] + schedule_item[var]
        output_records.append(record)

output_columns = ['Product', 'Assigned Line', 'Metric'] + [f'Month {i+1}' for i in range(planning_horizon)]
output_df = pd.DataFrame(output_records, columns=output_columns)

# Exibindo o DataFrame
output_df

,Product,Assigned Line,Metric,Month 1,Month 2,Month 3
0,0,2,Initial Inventory,36.0,72.0,144.0
1,0,2,Production,71.0,105.0,114.0
2,0,2,Demand,35.0,33.0,47.0
3,0,2,Final Inventory,72.0,144.0,211.0
4,1,2,Initial Inventory,51.0,102.0,122.0
...,...,...,...,...,...,...
75,18,2,Final Inventory,65.0,38.0,0.0
76,19,2,Initial Inventory,33.0,16.0,0.0
77,19,2,Production,0.0,0.0,0.0
78,19,2,Demand,17.0,18.0,24.0


In [9]:
output_df.to_excel('output/production_plan.xlsx', index=False)